# TensorFlow essentials

### 1. Import TensorFlow and Numpy:

In [1]:
import tensorflow as tf
import numpy as np

### 2. Defining tensors

A tensor is a generalization of a matrix that specifies an element by an arbitrary number of indices.
The syntax for tensors is even more nested vectors. For example, a 2-by-3-by-2 tensor is [[[1,2], [3,4], [5,6]], [[7,8], [9,10], [11,12]]], which can be thought of as two matrices, each of size 3-by-2. Consequently, we say this tensor has a rank of 3.

In [7]:
m1 = [[1.0, 2.0], 
      [3.0, 4.0]]

m2 = np.array([[1.0, 2.0], 
               [3.0, 4.0]], dtype=np.float32)

m3 = tf.constant([[1.0, 2.0], 
                  [3.0, 4.0]])

# Let's see what happens when we print them:
print(type(m1))
print(type(m2))
print(type(m3))

# By the way, there's a function called convert_to_tensor(...) that does exactly what you might expect.
# Let's use it to create tensor objects out of various types:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)

print(type(t1))
print(type(t2))
print(type(t3))

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


### 3. Evaluating ops 

In [11]:
x = tf.constant([[1, 2]])             # Start with a 1x2 matrix

neg_x = tf.negative(x)                # Let's negate it
print(neg_x)                          # It doesn't even perform the negation computation if you print.

with tf.Session() as sess:            # You need to summon a session so you can launch the negation op:
    result = sess.run(neg_x)
    print(result)

Tensor("Neg_5:0", shape=(1, 2), dtype=int32)
[[-1 -2]]


### 4. Interactive session

In [12]:
sess = tf.InteractiveSession()       # Interactive sessions are another way to use a session.

# We have a matrix we want to invert:
x = tf.constant([[1., 2.]])
neg_op = tf.negative(x)

# Since we're using an interactive session, we can just call the eval() method on the op.

result = neg_op.eval()
print(result)

sess.close()

[[-1. -2.]]


### 5. Session logging

In [10]:
x = tf.constant([[1, 2]])
neg_op = tf.negative(x)

# Start the session with a special config passed in to the constructor to enable logging
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_op)
    print(result)

[[-1 -2]]


### 6. Using variables

In [13]:
sess = tf.InteractiveSession()

# Below is a series of numbers. Don't worry what they mean. Just for fun, let's think of them as neural activations.
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]

spike = tf.Variable(False)          # Create a boolean variable called spike to detect a sudden increase in the values.
spike.initializer.run()             # All variables must be initialized, so do that by calling run() on its initializer

# Loop through the data and update the spike variable when there is a significant increase:
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, tf.constant(True))
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    print("Spike", spike.eval())

sess.close()

Spike False
Spike True
Spike False
Spike False
Spike True
Spike False
Spike True


### 7. Saving and Loading variables

In [2]:
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  inc_v1.op.run()
  dec_v2.op.run()
  # Save the variables to disk.
  save_path = saver.save(sess, "./model.ckpt")
  print("Model saved in file: %s" % save_path)



tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "./model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

Model saved in file: ./model.ckpt
INFO:tensorflow:Restoring parameters from ./model.ckpt
Model restored.
v1 : [ 1.  1.  1.]
v2 : [-1. -1. -1. -1. -1.]


### 8. Using TensorBoard

In [1]:
import tensorflow as tf
import numpy as np

# Create a vector of 1000 numbers with a mean of 10 and standard deviation of 1
raw_data = np.random.normal(10, 1, 100)


alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.)


update_avg = alpha * curr_value + (1 - alpha) * prev_avg

# Create a summary node for the averages & values
avg_hist = tf.summary.scalar("running_average", update_avg)
value_hist = tf.summary.scalar("incoming_values", curr_value)

#Merge the summaries to make it easier to run t ogether
merged = tf.summary.merge_all()

# Pass in the “logs” directory location to the writer
writer = tf.summary.FileWriter("./logs")


init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        summary_str, curr_avg = sess.run([merged, update_avg], feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        print(raw_data[i], curr_avg)
        
        # Add the summary to the writer
        writer.add_summary(summary_str, i)


# use following command to run tenserboard:
# tensorboard --logdir=./logs

9.47635434563 0.473818
9.60296265725 0.930275
9.85807458641 1.37666
10.5604578661 1.83585
9.4838370467 2.21825
10.7911589578 2.6469
9.45528987947 2.98732
7.88915009735 3.23241
8.50221899224 3.4959
9.85582043906 3.8139
10.0551507106 4.12596
9.23608385175 4.38146
9.76630070154 4.65071
10.4291219809 4.93963
8.3545398698 5.11037
9.68754538127 5.33923
9.09922122021 5.52723
10.9397857317 5.79786
9.68718500314 5.99233
10.675258966 6.22647
9.40013638064 6.38516
10.1726575743 6.57453
11.3738005278 6.81449
8.81947069403 6.91474
8.81165201316 7.00959
10.1802751781 7.16812
8.13426583658 7.21643
11.7745697787 7.44434
10.3357449044 7.58891
8.61432080671 7.64018
9.33638093012 7.72499
8.35274831443 7.75637
10.3979211107 7.88845
10.4836790465 8.01821
10.3588084955 8.13524
10.1123917926 8.2341
11.087338253 8.37676
10.5259142536 8.48422
9.36115473557 8.52807
11.2844634711 8.66589
9.73973670829 8.71958
10.2316002506 8.79518
10.4828468294 8.87956
10.1835488592 8.94476
9.48677892302 8.97186
9.12667443988 8.

# 